In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
# Create a dash application
app = dash.Dash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    ],
                                    value='ALL',
                                    placeholder="place holder here",
                                    searchable=True
                                ),
                                html.Br(),
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
 
                                html.Div([dcc.Graph(id='success-pie-chart')]),
                                html.Br(),
                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=int(min_payload),
                                    max=int(max_payload),
                                    step=1000, 
                                    marks={
                                        int(min_payload): f'{int(min_payload)}',
                                        int(max_payload): f'{int(max_payload)}'
                                    },
                                    value=[int(min_payload), int(max_payload)]
                                ),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                               
])
                
                            
                            # TASK 4:
                                # Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
                            
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
    )
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        success_by_site = spacex_df.groupby('Launch Site', as_index=False)['class'].sum()
        fig = px.pie(
            success_by_site, 
            values='class', 
            names='Launch Site', 
            title='Successful Launches by Site')
        return fig
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site].copy()
        site_df['Outcome'] = site_df['class'].map({1: 'Success', 0 : 'Fail'})
        counts = site_df['Outcome'].value_counts().reset_index()
        counts.columns = ['Outcome', 'count']
        fig = px.pie(
            counts, 
            values = 'count',
            names = 'Outcome',
            title = f'Success vs Fail in {entered_site}',
            color = 'Outcome',
            color_discrete_map={'Success': '#2ca02c', 'Fail': '#d62728'}
        )
        return fig

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >=low) &
        (spacex_df['Payload Mass (kg)'] <=high)
    ].copy()
    if selected_site is not None and selected_site != 'ALL':
        df = df[df['Launch Site'] == selected_site]
    title = (
        f'Payload vs. Launch Outcome | Site: {selected_site} | Range: {low}-{high} kg'
        if selected_site != 'ALL'
        else f'Payload vs. Launch Outcome (All Sites) | Range: {low}-{high} kg'
    )
    fig = px.scatter(
        df,
        x='Payload Mass (kg)',
        y='class',
        color = 'Booster Version Category',
        title = title,
        hover_data=['Launch Site', 'class', 'Payload Mass (kg)']
    )
    return fig
                              
# Run the app
if __name__ == '__main__':
    app.run()
